In [ ]:
## Variables

In [18]:
srr_names <- c("SRR5090597", "SRR5090599")
donor_name <- "hpv16"
recipient_name <- "USCShg38"
inputs_folder <- "../../wallaby/workflows/cromwell-final-outputs/"
#inputs_folder <- "../../wallaby/workflows/datasets/hpv16/"
donor_ref_genome <- "../../wallaby/data/ref_genomes/hpv/HPV16.fasta"
recipient_ref_genome <- "../../wallaby/data/ref_genomes/human/USCS.hg38.fasta"

In [24]:
srr_names <- c("SRR12091998", "SRR12091997", "SRR12091996", "SRR12091995", "SRR12091994", "SRR12091993", "SRR12091992", "SRR12091991", 
"SRR12091990")
donor_name <- "sars-cov-2"
recipient_name <- "USCShg38"
inputs_folder <- "../../wallaby/workflows/datasets/sars-cov-2/"
donor_ref_genome <- "../../wallaby/data/ref_genomes/sars-cov-2/wuhan-hu-1.fasta"
recipient_ref_genome <- "../../wallaby/data/ref_genomes/human/USCS.hg38.fasta"

In [11]:
srr_names <- c("SRR11772358", "SRR11772359", "SRR11772360", "SRR11772361", "SRR11772362", "SRR11772363", "SRR11772364", "SRR11772365", 
"SRR11772366", "SRR11772367", "SRR11772368", "SRR11772369", "SRR11772370", "SRR11772371", "SRR11772372", "SRR11772373", 
"SRR11772374", "SRR11772375", "SRR11772376", "SRR11772377", "SRR11772378", "SRR11772379", "SRR11772380", "SRR11772381", 
"SRR11772382", "SRR11772383", "SRR11772384", "SRR11772385", "SRR11772386", "SRR11772387", "SRR11772388", "SRR11772389", 
"SRR11772390", "SRR11772391", "SRR11772392", "SRR11772393", "SRR11772394", "SRR12340057", "SRR12340058", "SRR12340059", 
"SRR12340060", "SRR12340061", "SRR12340062", "SRR12340063", "SRR12340064", "SRR12340065", "SRR12340066", "SRR12340067", 
"SRR12340068", "SRR12340069", "SRR12340070", "SRR12340071", "SRR12340072", "SRR12340073", "SRR12340074", "SRR12340075", 
"SRR12340076", "SRR12340077", "SRR12340078", "SRR12340079", "SRR12340080", "SRR12340081", "SRR12340082", "SRR12340083", 
"SRR12340084", "SRR12340085", "SRR12340086", "SRR12340087", "SRR12340088", "SRR12340089", "SRR12340090", "SRR12340091", 
"SRR12340092", "SRR12340093", "SRR12340094", "SRR12340095", "SRR12340096", "SRR12340097", "SRR12340098", "SRR12340099", 
"SRR12340100", "SRR12340101", "SRR12340102", "SRR12340103", "SRR12340104", "SRR12340105", "SRR12340106", "SRR12340107")
donor_name <- "sars-cov-2"
recipient_name <- "USCShg38"
inputs_folder <- "../../wallaby/workflows/datasets/sars-cov-2-biopsies/"
donor_ref_genome <- "../../wallaby/data/ref_genomes/sars-cov-2/wuhan-hu-1.fasta"
recipient_ref_genome <- "../../wallaby/data/ref_genomes/human/USCS.hg38.fasta"

In [ ]:
# HPV16
"SRR5090597", "SRR5090599"

# SARS-COV-2
"SRR12091998", "SRR12091997", "SRR12091996", "SRR12091995", "SRR12091994", "SRR12091993", "SRR12091992", "SRR12091991", 
"SRR12091990"

"SRR12507513", "SRR12507514", "SRR12507515", "SRR12507516", "SRR12507517", "SRR12507518", "SRR12507519", 
"SRR12507520", "SRR12507521", "SRR12507522", "SRR12507523", "SRR12507524", "SRR12507525", "SRR12507526", "SRR12507527", 
"SRR12507528", "SRR12507529", "SRR12507530", "SRR12507531", "SRR12507532", "SRR12507533", "SRR12507534", "SRR12507535", 
"SRR12507536", "SRR12507537", "SRR12507538", "SRR12507539", "SRR12507540", "SRR12507541", "SRR12507542", "SRR12507543", 
"SRR12507544", "SRR12507545", "SRR12507546", "SRR12507547", "SRR12507548", "SRR12507549", "SRR12507550", "SRR12507551", 
"SRR12507552"

"SRR12134528", "SRR12134529", "SRR12134530", "SRR12134531", "SRR12134532", "SRR12134533", "SRR12134534", 
"SRR12134535", "SRR12134536", "SRR12134537", "SRR12134538", "SRR12134539", "SRR12134540", "SRR12134541", "SRR12134542", 
"SRR12134543", "SRR12134544", "SRR12134545"

"SRR11772358", "SRR11772359", "SRR11772360", "SRR11772361", "SRR11772362", "SRR11772363", "SRR11772364", "SRR11772365", 
"SRR11772366", "SRR11772367", "SRR11772368", "SRR11772369", "SRR11772370", "SRR11772371", "SRR11772372", "SRR11772373", 
"SRR11772374", "SRR11772375", "SRR11772376", "SRR11772377", "SRR11772378", "SRR11772379", "SRR11772380", "SRR11772381", 
"SRR11772382", "SRR11772383", "SRR11772384", "SRR11772385", "SRR11772386", "SRR11772387", "SRR11772388", "SRR11772389", 
"SRR11772390", "SRR11772391", "SRR11772392", "SRR11772393", "SRR11772394", "SRR12340057", "SRR12340058", "SRR12340059", 
"SRR12340060", "SRR12340061", "SRR12340062", "SRR12340063", "SRR12340064", "SRR12340065", "SRR12340066", "SRR12340067", 
"SRR12340068", "SRR12340069", "SRR12340070", "SRR12340071", "SRR12340072", "SRR12340073", "SRR12340074", "SRR12340075", 
"SRR12340076", "SRR12340077", "SRR12340078", "SRR12340079", "SRR12340080", "SRR12340081", "SRR12340082", "SRR12340083", 
"SRR12340084", "SRR12340085", "SRR12340086", "SRR12340087", "SRR12340088", "SRR12340089", "SRR12340090", "SRR12340091", 
"SRR12340092", "SRR12340093", "SRR12340094", "SRR12340095", "SRR12340096", "SRR12340097", "SRR12340098", "SRR12340099", 
"SRR12340100", "SRR12340101", "SRR12340102", "SRR12340103", "SRR12340104", "SRR12340105", "SRR12340106", "SRR12340107"

# Tertiary analysis of a Donor-to-Recipient pipeline

In [4]:
## Setup Environment

In [2]:
# Enable multithreading when possible (library dependent)
options(Ncpus = parallel::detectCores())
Sys.setenv(OMP_NUM_THREADS=toString(parallel::detectCores()))
Sys.setenv(OMP_THREAD_LIMIT=toString(parallel::detectCores()))
Sys.setenv(OMP_NUM_THREADS=parallel::detectCores())
Sys.setenv(OMP_THREAD_LIMIT=parallel::detectCores())

# Install tricky packages
suppressMessages(install.packages("../../BSgenome.Hsapiens.UCSC.hg38_1.4.3.tar.gz", repos = NULL, type = "source"))
suppressPackageStartupMessages(library(BSgenome.Hsapiens.UCSC.hg38))

## Load or install and load all libraries
suppressPackageStartupMessages(library("pacman", character.only = TRUE))

# List of CRAN packages to either Load, or Install and Load
pacman::p_load(dplyr, 
               ggplot2, shiny, shinyLP, DT,  ggrepel,  tidyr, data.table, 
               kableExtra, knitr, IRdisplay)

# List of Bioconductor packages to either Load, or Install and Load
pacman::p_load(GenomicFeatures, GenomicAlignments,  Rsubread,  Rsamtools, bamsignals,  
               rtracklayer, GenomicRanges, org.Hs.eg.db, Organism.dplyr,
               TxDb.Hsapiens.UCSC.hg38.knownGene,  regioneR, karyoploteR,  seqinr, Repitools, Gviz, Biostrings)

In [3]:
## Helper Functions

In [4]:
## Loading pipeline output files
# Try to open all files - note that some will not exists as that specific crossing bucket did not return any matches

In [25]:
try(rm(srrs_donor))
try(rm(srrs_recipient))

In [26]:
srrs_donor <- list()
srrs_recipient <- list()

for (srr_name in srr_names){
    file <- paste(inputs_folder, paste(srr_name, '-to-', donor_name, sep = ""),"_MMd_MUr.bed", sep = "")
    if (file.exists(file)) {donor_MMd_MUr <- import(file)}

    file <- paste(inputs_folder, paste(srr_name, '-to-', donor_name, sep = ""),"_MMd_UMr.bed", sep = "")
    if (file.exists(file)) {donor_MMd_UMr <- import(file)}

    file <- paste(inputs_folder, paste(srr_name, '-to-', donor_name, sep = ""),"_MUd_MMr.bed", sep = "")
    if (file.exists(file)) {donor_MUd_MMr <- import(file)}

    file <- paste(inputs_folder, paste(srr_name, '-to-', donor_name, sep = ""),"_MUd_UMr.bed", sep = "")
    if (file.exists(file)) {donor_MUd_UMr <- import(file)}

    file <- paste(inputs_folder, paste(srr_name, '-to-', donor_name, sep = ""),"_UMd_MMr.bed", sep = "")
    if (file.exists(file)) {donor_UMd_MMr <- import(file)}

    file <- paste(inputs_folder, paste(srr_name, '-to-', donor_name, sep = ""),"_UMd_MUr.bed", sep = "")
    if (file.exists(file)) {donor_UMd_MUr <- import(file)}
    
    file <- paste(inputs_folder, paste(srr_name, '-to-', recipient_name, sep = ""),"_MMd_MUr.bed", sep = "")
    if (file.exists(file)) {recip_MMd_MUr <- import(file)}

    file <- paste(inputs_folder, paste(srr_name, '-to-', recipient_name, sep = ""),"_MMd_UMr.bed", sep = "")
    if (file.exists(file)) {recip_MMd_UMr <- import(file)}

    file <- paste(inputs_folder, paste(srr_name, '-to-', recipient_name, sep = ""),"_MUd_MMr.bed", sep = "")
    if (file.exists(file)) {recip_MUd_MMr <- import(file)}

    file <- paste(inputs_folder, paste(srr_name, '-to-', recipient_name, sep = ""),"_MUd_UMr.bed", sep = "")
    if (file.exists(file)) {recip_MUd_UMr <- import(file)}

    file <- paste(inputs_folder, paste(srr_name, '-to-', recipient_name, sep = ""),"_UMd_MMr.bed", sep = "")
    if (file.exists(file)) {recip_UMd_MMr <- import(file)}

    file <- paste(inputs_folder, paste(srr_name, '-to-', recipient_name, sep = ""),"_UMd_MUr.bed", sep = "")
    if (file.exists(file)) {recip_UMd_MUr <- import(file)}

    donor_beds <- list()
    donor_names <- list("MMd_MUr", "MMd_UMr", "MUd_MMr", "MUd_UMr", "UMd_MMr", "UMd_MUr")

    for (name in donor_names) {
        object_name <- paste("donor_", name, sep="")
        try(donor_beds[[name]] <- get(object_name))
    }
    srrs_donor[srr_name] <- list(donor_beds)
    
    recip_beds <- list()
    recip_names <- list("MMd_MUr", "MMd_UMr", "MUd_MMr", "MUd_UMr", "UMd_MMr", "UMd_MUr")

    for (name in recip_names) {
        object_name <- paste("recip_", name, sep="")
        try(recip_beds[[name]] <- get(object_name))
    }
    srrs_recipient[srr_name] <- list(recip_beds)
}

In [4]:
src <- suppressMessages(src_organism("TxDb.Hsapiens.UCSC.hg38.knownGene"))

Warning message:
"Ignoring remaining part of query: CREATE INDEX IF NOT EXISTS entrez_accession ON id_accession (entrez);"
Warning message:
"Ignoring remaining part of query: CREATE INDEX IF NOT EXISTS entrez_protein ON id_protein (entrez);"


In [27]:
summary_table <- function(table_type, srr = "number", granges, granges_labels, min_num_crossings = 3, min_num_reads = 3){
    granges_df <- lapply(granges, annoGR2DF)
    names(granges_df) <- granges_labels
    merged_df <- bind_rows(granges_df, .id = "crossing")
    merged_dt <- as.data.table(merged_df)

    ## Interval per chromosome
    merged_dt[,group := { ir <- IRanges(start, end); subjectHits(findOverlaps(ir, reduce(ir))) }, by = chr]
    merged_final <- merged_dt[, list(start=min(start), 
                                     stop=max(end), 
                                     num_crossings=length(unique(list(crossing)[[1]])),
                                     unique_crossings=list(unique(crossing)),
                                     num_reads=length(list(name)[[1]])
                                     ), by=list(group,chr)]
    merged_final <- merged_final[merged_final[, num_reads > (min_num_reads - 1)]]
    merged_final <- merged_final[merged_final[, num_crossings > (min_num_crossings - 1)]]
    
    if (table_type == "recipient") {
        txdb <- TxDb.Hsapiens.UCSC.hg38.knownGene
        Hsapiens <- BSgenome.Hsapiens.UCSC.hg38

        if (nrow(merged_final) != 0){
            merged_final$gene_name <- apply(merged_final, 1, FUN = function(x) toString(
                unique(unlist(suppressWarnings(annoGR2DF(
                                        transcripts(src, 
                                                     filter=~(GRangesFilter(
                                                         GenomicRanges::GRanges(
                                                             paste(toString(x["chr"]), ":", 
                                                                   as.integer(x["start"]), "-", 
                                                                   as.integer(x["stop"]), sep = "")))), 
                                                     columns=c("symbol")))$symbol)))))

            merged_final$srr <- srr

            merged_final$sequence <- apply(merged_final, 1, FUN = function(x) toString(getSeq(Hsapiens, 
                                                                               toString(x["chr"]), 
                                                                               start = as.integer(x["start"]), 
                                                                               end = as.integer(x["stop"]))))

            merged_final <- merged_final[, !"group"]
            merged_final <- merged_final[with(merged_final, order(num_reads, decreasing = TRUE)), ]

            return(merged_final)
        } else {
            return(data.table(chr="<NA>", start=0, stop=0, num_crossings=0, unique_crossings="<NA>", num_reads=0, gene_name="<NA>", srr=srr, sequence="<NA>"))
        }
    }                                   
    else if (table_type == "donor") {
        if (nrow(merged_final) != 0) {
            merged_final$srr <- srr
            merged_final <- merged_final[, !"group"]
            merged_final <- merged_final[with(merged_final, order(num_reads, decreasing = TRUE)), ]
            return(merged_final)
        } else {
            return(data.table(chr="<NA>", start=0, stop=0, num_crossings=0, unique_crossings="<NA>", num_reads=0, srr=srr))
        }
    }
}

In [28]:
srrs_list <- list()

for (srr_name in srr_names){
    srr_table <- summary_table(table_type = "donor", srr = srr_name, granges = srrs_donor[srr_name][[1]], names(srrs_donor[srr_name][[1]]), min_num_crossings = 4, min_num_reads = 10)
    srr_table$srr_name <- srr_name
    srrs_list[[srr_name]] <- srr_table
}

srrs_table = do.call(rbind, c(srrs_list, fill=TRUE))
srrs_table <- srrs_table[, !"srr_name"]

In [29]:
srrs_table %>%
kable("html") %>%
    kable_styling(bootstrap_options = "striped", full_width = F, position = "left") %>%
    kable_paper(full_width = F) %>%
    column_spec(7, width = "30em", width_max = "30em") %>%
    as.character() %>%
    display_html()     

chr,start,stop,num_crossings,unique_crossings,num_reads,srr
MN908947.3,24310,29903,6,"MMd_MUr, MMd_UMr, MUd_MMr, MUd_UMr, UMd_MMr, UMd_MUr",9208,SRR12091998
MN908947.3,3,113,4,"MMd_UMr, MUd_MMr, UMd_MMr, UMd_MUr",73,SRR12091998
MN908947.3,21804,22522,4,"MMd_MUr, MMd_UMr, UMd_MMr, UMd_MUr",51,SRR12091998
MN908947.3,23447,24195,4,"MMd_MUr, MMd_UMr, UMd_MMr, UMd_MUr",46,SRR12091998
MN908947.3,21540,21801,4,"MMd_MUr, MMd_UMr, UMd_MMr, UMd_MUr",28,SRR12091998
MN908947.3,22558,22712,4,"MMd_MUr, MMd_UMr, UMd_MMr, UMd_MUr",12,SRR12091998
MN908947.3,24925,29903,6,"MMd_MUr, MMd_UMr, MUd_MMr, MUd_UMr, UMd_MMr, UMd_MUr",8942,SRR12091997
MN908947.3,24298,24857,4,"MMd_MUr, MMd_UMr, UMd_MMr, UMd_MUr",41,SRR12091997
MN908947.3,24777,29903,6,"MMd_MUr, MMd_UMr, MUd_MMr, MUd_UMr, UMd_MMr, UMd_MUr",7764,SRR12091996
MN908947.3,133,418,4,"MMd_MUr, MMd_UMr, UMd_MMr, UMd_MUr",12,SRR12091996


In [30]:
srrs_list <- list()

for (srr_name in srr_names){
    srr_table <- summary_table(table_type = "recipient", srr = srr_name, granges = srrs_recipient[srr_name][[1]], names(srrs_recipient[srr_name][[1]]), 
                               min_num_crossings = 3, min_num_reads = 1)
    srr_table$srr_name <- srr_name
    srrs_list[[srr_name]] <- srr_table
}

srrs_table = do.call(rbind, c(srrs_list, fill=TRUE))
srrs_table <- srrs_table[, !"srr_name"]

In [31]:
srrs_table %>%
kable("html") %>%
    kable_styling(bootstrap_options = "striped", full_width = F, position = "left") %>%
    kable_paper(full_width = F) %>%
    column_spec(9, width = "30em", width_max = "30em") %>%
    as.character() %>%
    display_html()     

chr,start,stop,num_crossings,unique_crossings,num_reads,gene_name,srr,sequence
chr1,58783686,58784059,3,"MMd_MUr, UMd_MMr, UMd_MUr",41,JUN,SRR12091998,TCTGCCACTTGTCTCCGGTCCTCCCAGCCGGGAAAGTTCTTTGCTGCTGCAGCCGCTCGCCGACTTCCCCCGGCGGGCGCGTGGGTACCGCTGCTTTCCGCCGCTGTCAACAGCGCCTGGGCAGCAGGGCTCTCCTCCCGGGGGCGGCTGGAGACCAGGCTCTCTGGACACTCCCGAAACACCAGCCCGGGAGCCACAGGCGCTAGCTCTGGGCAGTTAGAGAGAAGGTGAAAAGAAAATAAGATTTGCAGTTCGGACTATACTGCCGACCTGGCTGGCTGGCTGTGTCTGTCTGTCTGCCTGACTCCGCGCACCTCCACTCCCGCCTCGCTGCTTCAGCCACACTCAGTGCAACTCTGAGCCCTTATCCAGCC
chr5,172770851,172771188,3,"MMd_MUr, UMd_MMr, UMd_MUr",27,DUSP1,SRR12091998,AGCGAAGAAGGAGCGGCAGTCCAGCAGCAGGCATTGCGCCGCTCGCTCCCCCAGCAGCGCCCGCAGGCCTCCAGCGTCCAGGGTGCCCACTTCCATGACCATGGCCGGCCTCAGCGCCCCCAGCGTGATCGGCCCTGCGGTGCTCTTTGTCTGTTCTCGGGGCCAAGGGCAGGGCGGCGCTTTTCGAGGAAAAGCTAGACCCCCGGGTCTCTCTGCGCCGAACCAAAAGCCGCTTTTGGACTGAGAGAGGAGCGTCACGCGGGGCTCCGGGCTCCTCGGCTTCTTCGCGGTTCCCCCGACTGCCCCTCCGACCCGCGTCGCACACACAGCCCAAATGT
chr9,125241116,125241339,3,"MMd_MUr, UMd_MMr, UMd_MUr",21,HSPA5,SRR12091998,GAGAGCTTCATCTTGCCAGCCAGTTGGGCAGCAGCAGGCAGTCCAGCCACAGGCCGTAGCACAGGAGCACAGCGCAATTTCCGACTTGCAGGCGGCAGGGGCCCGGGGTCACAAGGCGCCACGAACCAGGCGAAGGGCAGGTCTAGAAATACAGGCCGCGGCGCTTCCCTCTCACACTCGCGAAACACCCCAATAGGTCAATCTGTCTGTGCTGTCTTGGCCGG
chr2,85667678,85667807,3,"MMd_MUr, UMd_MMr, UMd_MUr",7,SFTPB,SRR12091998,CGGCTCCCACATGTCCCCAGACTTCCTGTAGGCAATGCCCTAGGGCTCTGCACTGCAATGCTTGCTCCAGGCTTTGGCACCAGAACTCAGGGCCCTGGGCACAGGCCAAGGATGAGGTGGTCCAGGCAGC
chr5,172770898,172771189,3,"MMd_MUr, UMd_MMr, UMd_MUr",47,DUSP1,SRR12091997,CCCCCAGCAGCGCCCGCAGGCCTCCAGCGTCCAGGGTGCCCACTTCCATGACCATGGCCGGCCTCAGCGCCCCCAGCGTGATCGGCCCTGCGGTGCTCTTTGTCTGTTCTCGGGGCCAAGGGCAGGGCGGCGCTTTTCGAGGAAAAGCTAGACCCCCGGGTCTCTCTGCGCCGAACCAAAAGCCGCTTTTGGACTGAGAGAGGAGCGTCACGCGGGGCTCCGGGCTCCTCGGCTTCTTCGCGGTTCCCCCGACTGCCCCTCCGACCCGCGTCGCACACACAGCCCAAATGTC
chr1,58783626,58784059,3,"MMd_MUr, UMd_MMr, UMd_MUr",45,JUN,SRR12091997,CTTTACCGCCGTGGAGAAGCCTAAGACGCAGGAAAGGCTTGCAAAAGTTGGCTCCGGGACTCTGCCACTTGTCTCCGGTCCTCCCAGCCGGGAAAGTTCTTTGCTGCTGCAGCCGCTCGCCGACTTCCCCCGGCGGGCGCGTGGGTACCGCTGCTTTCCGCCGCTGTCAACAGCGCCTGGGCAGCAGGGCTCTCCTCCCGGGGGCGGCTGGAGACCAGGCTCTCTGGACACTCCCGAAACACCAGCCCGGGAGCCACAGGCGCTAGCTCTGGGCAGTTAGAGAGAAGGTGAAAAGAAAATAAGATTTGCAGTTCGGACTATACTGCCGACCTGGCTGGCTGGCTGTGTCTGTCTGTCTGCCTGACTCCGCGCACCTCCACTCCCGCCTCGCTGCTTCAGCCACACTCAGTGCAACTCTGAGCCCTTATCCAGCC
chr9,125241036,125241337,3,"MMd_MUr, UMd_MMr, UMd_MUr",17,HSPA5,SRR12091997,CCACCGTGCCCACGTCCTCCTTCTTGTCCTCCTCCTCGGCCCGCGCCGCGCTGAGCAGCAGCAGCATCGCGGCCACCAGGGAGAGCTTCATCTTGCCAGCCAGTTGGGCAGCAGCAGGCAGTCCAGCCACAGGCCGTAGCACAGGAGCACAGCGCAATTTCCGACTTGCAGGCGGCAGGGGCCCGGGGTCACAAGGCGCCACGAACCAGGCGAAGGGCAGGTCTAGAAATACAGGCCGCGGCGCTTCCCTCTCACACTCGCGAAACACCCCAATAGGTCAATCTGTCTGTGCTGTCTTGGCC
chr2,151010105,151010155,3,"MMd_MUr, MUd_MMr, UMd_MMr",4,,SRR12091997,TTTCTTTCTTTTATTTTTATTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT
chr5,172770894,172771188,3,"MMd_MUr, UMd_MMr, UMd_MUr",36,DUSP1,SRR12091996,CGCTCCCCCAGCAGCGCCCGCAGGCCTCCAGCGTCCAGGGTGCCCACTTCCATGACCATGGCCGGCCTCAGCGCCCCCAGCGTGATCGGCCCTGCGGTGCTCTTTGTCTGTTCTCGGGGCCAAGGGCAGGGCGGCGCTTTTCGAGGAAAAGCTAGACCCCCGGGTCTCTCTGCGCCGAACCAAAAGCCGCTTTTGGACTGAGAGAGGAGCGTCACGCGGGGCTCCGGGCTCCTCGGCTTCTTCGCGGTTCCCCCGACTGCCCCTCCGACCCGCGTCGCACACACAGCCCAAATGT
chr1,58783803,58784065,3,"MMd_MUr, UMd_MMr, UMd_MUr",27,JUN,SRR12091996,TGGGCAGCAGGGCTCTCCTCCCGGGGGCGGCTGGAGACCAGGCTCTCTGGACACTCCCGAAACACCAGCCCGGGAGCCACAGGCGCTAGCTCTGGGCAGTTAGAGAGAAGGTGAAAAGAAAATAAGATTTGCAGTTCGGACTATACTGCCGACCTGGCTGGCTGGCTGTGTCTGTCTGTCTGCCTGACTCCGCGCACCTCCACTCCCGCCTCGCTGCTTCAGCCACACTCAGTGCAACTCTGAGCCCTTATCCAGCCCGAGCT


In [ ]:
transcripts(src, columns=c("symbol"))

In [ ]:
# get GFF from covid and convert to granges - make sure it's got the gene names there

In [ ]:
# Then find how to extract sequence length from granges - boom